# Identifier les textes évoquant le concept de république

TODO:
pb ici : vérifier d'ou vient mais
- changement taille fichier entrée (check)
- et (par conséquent ?) parfois match_valide 4900/4592 et match_simple 4500/4114.

Logique de la tentative :
- regex
- mais exclure certains termes
- mais comme les termes exclus peuvent apparaitre aussi avec les termes voulu, éviter de chainer et finir par virer des trucs qu'on aurait voulu (les idées républicaines sont menacées par Les Républicains)
- Mais comme je suis pas bien sur de moin tentative GPT à vérifier

In [9]:
import pandas as pd

df = pd.read_csv("../data/data_cleaning.csv", low_memory=False)

In [10]:
df.shape

(129769, 33)

In [11]:
import re

# Regex du champ lexical République (simplifié ici)
# pattern_lexical = re.compile(r"\b(anti-?république(s)?|république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b", re.I)
# sinon :
pattern_lexical = re.compile(
    r"(?<!\w)(anti-?|pré-?|post-?)?(république(s)?|républicain(e)?s?|républicanisme|républicanis(er|ation))\b",
    re.I,
)


# Regex des expressions à exclure
pattern_excl = re.compile(
    r"\bLes Républicains\b|Président de la République"
)  # ici garder la casse


def contains_lexical_outside_excl(text):
    # Trouve toutes les positions des expressions exclues
    excl_positions = [m.span() for m in pattern_excl.finditer(text)]

    # Fonction pour vérifier si une position est dans une zone exclue
    def in_excl(pos):
        for start, end in excl_positions:
            if start <= pos < end:
                return True
        return False

    # Chercher toutes les occurences du champ lexical
    for match in pattern_lexical.finditer(text):
        start_pos = match.start()
        if not in_excl(start_pos):
            return True
    return False


# Appliquer sur la colonne
df["match_valide"] = df["Texte"].apply(contains_lexical_outside_excl)

In [12]:
df[df["match_valide"]].shape

(4592, 34)

In [13]:
# Version bourrin
# Champ lexical : True si le texte contient un terme du champ lexical
lexical_mask = df["Texte"].str.contains(pattern_lexical)

# Exclusion : True si le texte contient une expression à exclure
excl_mask = df["Texte"].str.contains(pattern_excl)

# On garde ceux qui matchent lexicalement mais ne contiennent pas d'exclusion
df["match_simple"] = lexical_mask & ~excl_mask


/var/folders/rq/xsj46x_s2rg87wdksm1_jl3c0000gn/T/ipykernel_30899/2994468294.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  lexical_mask = df["Texte"].str.contains(pattern_lexical)


In [14]:
df[df["match_simple"]].shape

(4114, 35)

In [15]:
df_match = df[df["match_valide"]]
df_match.to_csv("../data/df_match.csv", index=False)